# Testing different embedding schemes for information retrieval
## Step 1: Load sample text

In [1]:
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from src.utils import read_txt, split_txt, aiap_qna_quickscore

/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
text = read_txt('../data/fund_guide_extend.txt')
# text = [x.decode("utf-8") for x in text]

In [3]:
condition_terms = split_txt(text, qa=False)
print(len(condition_terms))
# condition_terms[0]

22


# Step 2: Load sample questions

In [4]:
import pandas as pd
df_queries = pd.read_csv('../data/Consolidated emails.csv', encoding='iso-8859-1')
print(len(df_queries))

73


In [7]:
# Answer collection
initial_list=df_queries['Email Queries'].values
modified_list=[val for val in initial_list for _ in (0, 1, 2)]
df_comparisons = pd.DataFrame({'queries' : modified_list})

# Step 3: define similarity function
The cosine similarity function returns the cosine similarity given a query string, an encoder, and an array of knowledgebase embeddings

In [7]:
def cosine_sim_results(query_str, encoder, kb_embeddings, **kwargs):
    if kwargs:
        qn_embedding = encoder(query_str, kwargs.get('tokenize', None))
    else:
        qn_embedding = encoder(query_str)
    results = cosine_similarity(kb_embeddings, qn_embedding)
    return results

# Utility functions

In [9]:
def dummy_embed_fn(vector):
    return vector

# Exp 1: Test InferSent model

In [10]:
from InferSent.models import InferSent
import torch
V = 1
MODEL_PATH = '../encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [11]:
W2V_PATH = '../fastText/crawl-300d-2M.vec'
infersent.set_w2v_path(W2V_PATH)

In [12]:
infersent.build_vocab(condition_terms, tokenize=True)

Found 2646(/2721) words with w2v vectors
Vocab size : 2646


In [13]:
infersent.update_vocab(condition_terms, tokenize=True)

Found 0(/75) words with w2v vectors
New vocab size : 2646 (added 0 words)


In [14]:
response_results = infersent.encode(condition_terms, tokenize=True)
questions = list(df_queries['Email Queries'])
question_results = infersent.encode(questions, tokenize=True)

In [15]:
print(response_results.shape, question_results.shape)

(148, 4096) (73, 4096)


In [16]:
# sample question
# question=['how frequent can i get disbursements?']
# condition_terms[cosine_sim_results(question, infersent.encode, embeddings, tokenize=True).argmax()]

In [17]:
# question=['who is RESPONSIBLE FOR DATA CHARGES?']
# sortargs=cosine_sim_results(question, infersent.encode, embeddings, tokenize=True).argsort(axis=0)
# print(sortargs.shape)
# for ii,arg in enumerate(sortargs[::-1,0]):
#     print(ii, condition_terms[arg])
#     if ii==4:
#         break

In [18]:
# list(cosine_sim_results(question, infersent.encode, embeddings, tokenize=True).argsort(axis=0).shape[-2:])

## Test with example queries

In [49]:
responses=[]
for index, ii in enumerate(df_queries.iterrows()):
    print('QN: ', ii[1]['Email Queries'])
    sortargs=np.flip(cosine_sim_results(question_results[index].reshape(1, -1), dummy_embed_fn, response_results).argsort(axis=0))
    for ans in range(3):
        responses.append(condition_terms[sortargs[ans,0]])
        print('ANS: ', condition_terms[sortargs[ans,0]])
df_comparisons['infersent']=responses

In [20]:
# responses=[]
# for ii in df_queries.iterrows():
#     print('QN: ', ii[1]['Email Queries'])
#     sortargs=np.flip(cosine_sim_results([ii[1]['Email Queries']], infersent.encode, embeddings, tokenize=True).argsort(axis=0))
#     for ans in range(3):
#         print(sortargs[ans, 0])
#         responses.append(condition_terms[sortargs[ans,0]])
#         print('ANS: ', condition_terms[sortargs[ans,0]])
# #     answer = condition_terms[cosine_sim_results([ii[1]['Email Queries']], infersent.encode, embeddings, tokenize=True).argmax()]
# #     print('ANS: ', answer)
# #     responses.append(answer)
# #     print('\n')
# df_comparisons['infersent']=responses

# Exp 2: Google universal sentence encoder

In [21]:
# !pip install tensorflow-gpu
# !pip install tensorflow-hub
import tensorflow as tf
import tensorflow_hub as hub

/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [22]:
#download the model to local so it can be used again and again
# !mkdir google_use
# Download the module, and uncompress it to the destination folder. 
# !curl -L "https://tfhub.dev/google/universal-sentence-encoder-large/3?tf-hub-format=compressed" | tar -zxvC ./google_use

In [23]:
embed = hub.Module("../google_use")

Instructions for updating:
Colocations handled automatically by placer.


W0913 07:56:00.905652 140286982317824 deprecation.py:323] From /anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [24]:
def use_embed(terms):
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        message_embeddings = session.run(embed(terms))
    return message_embeddings

# to only load session once.
# def embed_useT(module):
#     with tf.Graph().as_default():
#         sentences = tf.placeholder(tf.string)
#         embed = hub.Module(module)
#         embeddings = embed(sentences)
#         session = tf.train.MonitoredSession()
#     return lambda x: session.run(embeddings, {sentences: x})


In [25]:
response_results = use_embed(condition_terms)
all_questions = df_queries['Email Queries']
questions=list(all_questions)
question_results = use_embed(questions)
print(response_results.shape, question_results.shape)

InternalError: Dst tensor is not initialized.
	 [[node checkpoint_initializer_20 (defined at /anaconda/envs/py35/lib/python3.5/site-packages/tensorflow_hub/native_module.py:407) ]]

Caused by op 'checkpoint_initializer_20', defined at:
  File "/anaconda/envs/py35/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda/envs/py35/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/anaconda/envs/py35/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/anaconda/envs/py35/lib/python3.5/site-packages/tornado/platform/asyncio.py", line 127, in start
    self.asyncio_loop.run_forever()
  File "/anaconda/

In [ ]:
# responses=[]
# for ii in df_queries.iterrows():
#     print('QN: ', ii[1]['Email Queries'])
#     answer = condition_terms[cosine_sim_results([ii[1]['Email Queries']], use_embed, message_embeddings).argmax()]
#     print('ANS: ', answer)
#     responses.append(answer)
#     print('\n')
# df_comparisons['use']=responses

In [ ]:

responses=[]
for index, ii in enumerate(df_queries.iterrows()):
    print('QN: ', ii[1]['Email Queries'])
    sortargs=np.flip(cosine_sim_results(question_results[index].reshape(1, -1), dummy_embed_fn, response_results).argsort(axis=0))
    for ans in range(3):
        responses.append(condition_terms[sortargs[ans,0]])
        print('ANS: ', condition_terms[sortargs[ans,0]])
df_comparisons['use']=responses

In [ ]:
# responses=[]
# for ii in df_queries.iterrows():
#     print('QN: ', ii[1]['Email Queries'])
#     sortargs=np.flip(cosine_sim_results([ii[1]['Email Queries']], use_embed, message_embeddings).argsort(axis=0))
#     for ans in range(3):
#         responses.append(condition_terms[sortargs[ans,0]])
#         print('ANS: ', condition_terms[sortargs[ans,0]])
# df_comparisons['use']=responses

# Exp 3: Test the new QnA USE

In [ ]:
# !pip install sentencepiece
!pip install tf-sentencepiece

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tf_sentencepiece

In [ ]:
def init_model():
    # Set up graph.
    g = tf.Graph()
    with g.as_default():
        embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/1")
        # put placeholders
        question = tf.placeholder(dtype=tf.string, shape=[None])  # question
        response = tf.placeholder(dtype=tf.string, shape=[None])  # response
        response_context = tf.placeholder(dtype=tf.string, shape=[None])  # response context
        
        question_embeddings = embed(
        dict(input=question),
        signature="question_encoder", as_dict=True)

        response_embeddings = embed(
        dict(input=response,
             context=response_context),
        signature="response_encoder", as_dict=True)

        init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
    g.finalize()

    # Initialize session.
    session = tf.Session(graph=g, config=tf.ConfigProto(log_device_placement=True))
    session.run(init_op)
    return session, question_embeddings, response_embeddings, question, response, response_context

In [ ]:
# define inputs
responses = condition_terms
response_contexts = responses # no need to provide context
all_questions = df_queries['Email Queries']
questions=list(all_questions)

In [ ]:
session, question_embeddings, response_embeddings, question, response, response_context = init_model(questions, responses, response_contexts)

In [ ]:
# Compute embeddings.
response_results = session.run(response_embeddings)


In [ ]:
question_results = session.run(question_embeddings)
session.close()

In [ ]:
response_results['outputs'].shape

In [ ]:
# responses = []
# for index, ii in enumerate(df_queries.iterrows()):
#     print('QN: ', ii[1]['Email Queries'])
#     answer=condition_terms[cosine_sim_results(question_results['outputs'][index].reshape(1, -1), dummy_embed_fn, response_results['outputs']).argmax()]
#     print('ANS: ', answer)
#     responses.append(answer)
#     print('\n')
# df_comparisons['use_qa']=responses

In [ ]:
responses=[]
for index, ii in enumerate(df_queries.iterrows()):
    print('QN: ', ii[1]['Email Queries'])
    sortargs=np.flip(cosine_sim_results(question_results['outputs'][index].reshape(1, -1), dummy_embed_fn, response_results['outputs']).argsort(axis=0))
    for ans in range(3):
        responses.append(condition_terms[sortargs[ans,0]])
        print('ANS: ', condition_terms[sortargs[ans,0]])
df_comparisons['use_qa']=responses

# Save to csv

In [ ]:
df_comparisons.to_csv('../predictions_3_extend_batch2.csv', index=False)

# Test AIAP qna
7/10

In [2]:
from src.model import QnaEncoderModel
model=QnaEncoderModel()

/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Colocations handled automatically by placer.


W0920 06:57:38.990347 139906543216384 deprecation.py:323] From /anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0920 06:57:54.408357 139906543216384 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0920 06:58:00.903040 139906543216384 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [3]:
aiap_qa, aiap_questions = split_txt(read_txt('../data/aiap.txt'), qa=True)
# aiap_qa = split_txt(read_txt('./data/aiap.txt'))
print(aiap_qa[0], aiap_questions[0])

We are looking for candidates who possess a keen interest in the area of machine learning and data science. We believe that candidates can come from any area of specialisation, and our requirements are as follow: i)   Singaporean with a polytechnic diploma or university degree, ii) Proficient in Python or R and iii) Is able to implement Machine Learning Algorithms or have a background in Mathematics / Statistics / Computer Science.  Beyond that, demonstrated statistical fundamentals and programming ability will be helpful for the technical tests, but a keen learning attitude will be the most important to carry you through the programme.   Q1. WHAT SORT OF CANDIDATES ARE YOU LOOKING FOR?


In [4]:
answer_array = model.predict(aiap_qa)

In [5]:

#     return aiap_qa[(lambda x: cosine_similarity(answer_array, model.predict(x, type='query')))([question]).argmax()]
# aiap_qa[np.flip(cosine_sim_results(question_results['outputs'][index].reshape(1, -1), dummy_embed_fn, response_results['outputs']).argsort(axis=0))[:2]]

In [10]:

aiap_qna_quickscore(aiap_questions, answer_array, aiap_qa, model, 1)

0.5

In [50]:
for qn in aiap_context:
    print(qn, aiap_qna(qn, answer_arraym, aiap_qa, model, 1), '\n')

Q1. WHAT SORT OF CANDIDATES ARE YOU LOOKING FOR? (['Candidates can expect to be equipped in some or all of the following skills: data modelling/tuning, data engineering, data product-related software engineering, cloud applications. It ranges between individuals, but candidates can be adequately prepared in fields of data science, engineering and consultancy '], [3, 0, 6, 4, 9, 5, 8, 1, 2, 7]) 

Q2. WHAT WILL BE COVERED IN THE PROGRAMME? (['We will begin with some structured coursework involving programming, modelling and deployment. Following which, apprentices will be assigned to 100E projects to hone their skills in-depth, specific to each project. Generally, you will be doing the following: Deploy Machine Learning applications in line with industry best practices Select, train and tune Machine Learning algorithms Design and build Data Pipelines and Infrastructures for AI Use Cases '], [1, 7, 5, 0, 3, 6, 8, 9, 4, 2]) 

Q3. DO I HAVE TO PAY? (['If you drop out of the course, you will

## USE model
2/10

In [10]:
from src.model import USEModel
model=USEModel()
aiap_qa, aiap_context = split_txt(read_txt('../data/aiap.txt'), qa=True)
answer_array = model.predict(aiap_qa)
def aiap_qna(question):
    return aiap_qa[(lambda x: cosine_similarity(answer_array, model.predict(x)))([question]).argmax()]
for qn in aiap_context:
    print(qn, aiap_qna(qn), '\n')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0917 05:40:46.944837 140157478967040 saver.py:1483] Saver not created because there are no variables in the graph to restore


Q1. WHAT SORT OF CANDIDATES ARE YOU LOOKING FOR? Candidates can expect to be equipped in some or all of the following skills: data modelling/tuning, data engineering, data product-related software engineering, cloud applications. It ranges between individuals, but candidates can be adequately prepared in fields of data science, engineering and consultancy  

Q2. WHAT WILL BE COVERED IN THE PROGRAMME? YES. This is a FULL TIME programme where you will undergo training at an intensive pace over 9 months.  Due to our sponsorship requirements, you are not allowed to take no-pay leave to attend the programme.  

Q3. DO I HAVE TO PAY? If you drop out of the course, you will have to reimburse AISG the training stipends paid out to you and the fees for the programme.  

Q4. WHAT IS THE OUTCOME OF THIS PROGRAMME? YES. This is a FULL TIME programme where you will undergo training at an intensive pace over 9 months.  Due to our sponsorship requirements, you are not allowed to take no-pay leave to 

## Infersent
build vocab on context only: 1/10

In [12]:
from src.model import InferSent
model=InferSent()
aiap_qa, aiap_context = split_txt(read_txt('../data/aiap.txt'), qa=True)
model.build_vocab(aiap_context)
answer_array = model.predict(aiap_qa)
def aiap_qna(question):
    return aiap_qa[(lambda x: cosine_similarity(answer_array, model.predict(x)))([question]).argmax()]
for qn in aiap_context:
    print(qn, aiap_qna(qn), '\n')

Found 65(/67) words with w2v vectors
Vocab size : 65
Q1. WHAT SORT OF CANDIDATES ARE YOU LOOKING FOR? No payment is required. All apprentices will receive full sponsorship from AI Singapore and Infocomm and Media Development Authority.  The apprentices will also receive a stipend of between SGD$3,500 to $5,500/month depending on your working experience.  

Q2. WHAT WILL BE COVERED IN THE PROGRAMME? No payment is required. All apprentices will receive full sponsorship from AI Singapore and Infocomm and Media Development Authority.  The apprentices will also receive a stipend of between SGD$3,500 to $5,500/month depending on your working experience.  

Q3. DO I HAVE TO PAY? No payment is required. All apprentices will receive full sponsorship from AI Singapore and Infocomm and Media Development Authority.  The apprentices will also receive a stipend of between SGD$3,500 to $5,500/month depending on your working experience.  

Q4. WHAT IS THE OUTCOME OF THIS PROGRAMME? No payment is requi